In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling
### Associated lectures: Lectures 16 and 17

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
word1 = "phone"
word2 = "bowl"
word3 = "sky"
word4 = "car"

In [7]:
glove_wiki_vectors.most_similar(word1)

[('telephone', 0.9113394021987915),
 ('cellphone', 0.8122148513793945),
 ('phones', 0.8030568957328796),
 ('mobile', 0.7307462692260742),
 ('mail', 0.7292180061340332),
 ('internet', 0.7269454002380371),
 ('messages', 0.7129441499710083),
 ('wireless', 0.7091723084449768),
 ('telephones', 0.7049247622489929),
 ('cellular', 0.6969102621078491)]

In [8]:
glove_wiki_vectors.most_similar(word2)

[('bowls', 0.7603555917739868),
 ('super', 0.682817816734314),
 ('gator', 0.6623548865318298),
 ('nfl', 0.6438735723495483),
 ('bcs', 0.6366878151893616),
 ('playoffs', 0.6337442398071289),
 ('game', 0.6267698407173157),
 ('basketball', 0.6253952383995056),
 ('packers', 0.61924809217453),
 ('season', 0.611222505569458)]

In [9]:
glove_wiki_vectors.most_similar(word3)

[('skies', 0.7078207731246948),
 ('light', 0.6485515832901001),
 ('bright', 0.633054792881012),
 ('blue', 0.6260656714439392),
 ('dark', 0.6218482255935669),
 ('cloud', 0.6198915243148804),
 ('horizon', 0.6129894256591797),
 ('sun', 0.6126757860183716),
 ('lights', 0.6123525500297546),
 ('darkness', 0.6013585925102234)]

In [10]:
glove_wiki_vectors.most_similar(word4)

[('vehicle', 0.8630837798118591),
 ('truck', 0.8597878813743591),
 ('cars', 0.837166965007782),
 ('driver', 0.8185911178588867),
 ('driving', 0.781263530254364),
 ('motorcycle', 0.7553156614303589),
 ('vehicles', 0.7462257146835327),
 ('parked', 0.74594646692276),
 ('bus', 0.737270712852478),
 ('taxi', 0.7155269384384155)]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
# adapted from lecture notes 18
for pair in word_pairs:
    print(
        "The similarity between %s and %s is %0.3f"
        % (pair[0], pair[1], glove_wiki_vectors.similarity(pair[0], pair[1]))
    )

The similarity between coast and shore is 0.700
The similarity between clothes and closet is 0.546
The similarity between old and new is 0.643
The similarity between smart and intelligent is 0.755
The similarity between dog and cat is 0.880
The similarity between tree and lawyer is 0.077


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [13]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [14]:
def checkWordInWiki(word):
    try:
        glove_wiki_vectors[word]
        return True
    except:
        return False

In [15]:
for word in test_words:
    print(word, ":", checkWordInWiki(word))

covididiot : False
fomo : False
frenemies : True
anthropause : False
photobomb : False
selfie : False
pxg : False
pacg : False
cct : True
escc : True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [16]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [17]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [18]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283238

In [19]:
glove_wiki_vectors.similarity("white", "success")

0.35182396

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [20]:
analogy("father", "work", "mother")

father : work :: mother : ?


,Analogy word,Score
0,working,0.784925
1,well,0.732565
2,works,0.729545
3,done,0.723534
4,she,0.721947
5,so,0.693118
6,need,0.688049
7,how,0.686117
8,worked,0.682462
9,make,0.678711


In [21]:
glove_wiki_vectors.similarity("father", "work"), glove_wiki_vectors.similarity("mother", "work")

(0.5434319, 0.50242275)

In [22]:
analogy("rich", "smart", "poor")

rich : smart :: poor : ?


,Analogy word,Score
0,intelligent,0.567688
1,better,0.564519
2,automated,0.536484
3,stupid,0.531659
4,bad,0.526753
5,easier,0.524593
6,standard,0.519497
7,good,0.518949
8,care,0.514425
9,efficient,0.512467


In [23]:
glove_wiki_vectors.similarity("rich", "smart"), glove_wiki_vectors.similarity("poor", "smart")

(0.428874, 0.37506944)

In [24]:
analogy("male", "logical", "female")

male : logical :: female : ?


,Analogy word,Score
0,rational,0.662633
1,practical,0.650842
2,imperative,0.630941
3,explanation,0.629490
4,objective,0.623560
5,logic,0.622685
6,useful,0.616407
7,fundamental,0.612833
8,compelling,0.611380
9,straightforward,0.609975


In [25]:
glove_wiki_vectors.similarity("male", "logical"), glove_wiki_vectors.similarity("female", "logical")

(0.113570064, 0.08192001)

In [26]:
analogy("canadian", "kind", "french")

canadian : kind :: french : ?


,Analogy word,Score
0,sort,0.780599
1,nothing,0.700894
2,something,0.697036
3,nice,0.673517
4,always,0.668306
5,really,0.663018
6,thing,0.662657
7,so,0.659174
8,seems,0.658050
9,very,0.657290


In [27]:
glove_wiki_vectors.similarity("canadian", "kind"), glove_wiki_vectors.similarity("french", "kind")

(0.30371475, 0.38300937)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

1. There doesn't seem to be a lot of bias on my examples. The analogy gives similar words to what was queried, and the similarity scores are quite similar (differences are only 0.05 max). Though on the second example, 'rich : smart :: poor : ?', the analogy does show some negatively connotated vocabs like bad and stupid, which can be worrisome. It is implying that poor people are somewhat related to being bad or stupid like how rich people are related to smart, although the score is somewhat low.

2. An example would be using biased embeddings on job recruiting process. If there is a bias on certain gender stereotypes with a particular job position, the 'stronger' gender would be preferred over the other with similar skill set.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [28]:
from sklearn.datasets import fetch_20newsgroups

In [29]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [30]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [31]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [32]:
# adapted from lecture note 18, modified a little
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE", "NUM"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """

    clean_text = []

    for token in doc:
        if (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [33]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]

In [34]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,like subscribe leadership magazine wonder disk...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

- Check and exclude stopwords
- Check if the word meets minimum length of 2, exclude if <= 2
- Check and exclude word in certain part-of-speech and punctuations (adverb, pronoun, conjunction, particle, determiner, adposition, whitespace, numbers)
- lemmatization
- tokenization to lower case

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

Out of the 8 categories: 
['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast'
 ], I felt like they could be divided to 4 topics: 'sports', 'computer', 'religion' and 'politics'. So I'm choosing n_components=4. It also felt like the right number of topics that is not too many (would be too specific) nor too little (would be too broad)

 

In [35]:
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(df["text_pp"])

In [36]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 4
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42
)
document_topics = lda.fit_transform(X)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [37]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

In [38]:
import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
gun           play          file          god           
people        period        game          people        
armenian      pittsburgh    program       know          
turkish       detroit       use           think         
government    goal          edu           say           
state         det           image         come          
weapon        bos           run           time          
armenians     win           window        believe       
year          chicago       year          thing         
law           power         good          good          




- topic 0: politics
- topic 1: sports (also has city names of teams)
- topic 2: computer/programming
- topic 3: religion/beliefs

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [39]:
document_topics[:5]

array([[0.84018258, 0.00500876, 0.00522235, 0.14958631],
       [0.36241657, 0.00354547, 0.63030188, 0.00373608],
       [0.00446449, 0.00432122, 0.33914281, 0.65207148],
       [0.00915541, 0.20966619, 0.67791828, 0.10326012],
       [0.7864403 , 0.00582878, 0.00600888, 0.20172204]])

In [40]:
for i in range (5):
    print("Document %d is made up of ~%0.3f%% topic 0, ~%0.3f%% topic 1, ~%0.3f%% topic 2, and ~%0.3f%% topic 3."
          % (i, document_topics[i][0]*100, document_topics[i][1]*100, document_topics[i][2]*100, document_topics[i][3]*100))

Document 0 is made up of ~84.018% topic 0, ~0.501% topic 1, ~0.522% topic 2, and ~14.959% topic 3.
Document 1 is made up of ~36.242% topic 0, ~0.355% topic 1, ~63.030% topic 2, and ~0.374% topic 3.
Document 2 is made up of ~0.446% topic 0, ~0.432% topic 1, ~33.914% topic 2, and ~65.207% topic 3.
Document 3 is made up of ~0.916% topic 0, ~20.967% topic 1, ~67.792% topic 2, and ~10.326% topic 3.
Document 4 is made up of ~78.644% topic 0, ~0.583% topic 1, ~0.601% topic 2, and ~20.172% topic 3.


From the percentages, we can conclude that according to the model:
- Document 0 is 84% about politics (topic 0)
- Document 1 is 63% about computer (topic 2). According to the model, it also has 36% of politics (topic 0)
- Document 2 is 65% about religion (topic 3). It also has 34% of computer (topic 2)
- Document 3 is 67% about computer (topic 2), and 20% sports (topic 1)
- Document 4 is 78% about politics (topic 0), and 20% religion (topic 3)

In [41]:
df[:5]

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. wonder help u.s.c. provide pa...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...


As seen, the assigned topics (with highest %) matched the categories/target_name, except for Document 2 which is supposed to be about computer.

In [42]:
df["text"][2]

"\nActuallay I don't, but on the other hand I don't support the idea of having\none newsgroup for every aspect of graphics programming as proposed by Brian,\nin his reply to my original posting.\nI would suggest a looser structure more like a comp.graphics.programmer,\ncomp.graphics.hw_and_sw\nThe reason for making as few groups as possible is for the same reason you\nsay we shouldn't spilt up, not to get to few postings every day.\nI takes to much time to browse through all postings just to find two or \nthree I'm interested in.\n\nI understand and agree when you say you want all aspects of graphics in one\nmeeting. I agree to some extension. I see news as a forum to exchange ideas,\nhelp others or to be helped. I think this is difficult to achive if there\nare so many different things in one meeting.\n\nGood evening netters|-)"

In [43]:
df["text_pp"][2]

'actuallay hand support idea have newsgroup aspect graphic programming propose brian reply original posting suggest loose structure comp.graphics.programmer comp.graphics.hw_and_sw reason make group possible reason spilt posting day take time browse posting find interested understand agree want aspect graphic meeting agree extension news forum exchange idea help help think difficult achive different thing meeting good evening netters|-'

According to the model, this document is made up of ~65% topic 3, which I considered to be about religion/beliefs and only 34% computer, which is supposed to be the correct topic. This is probably because it doesn't use many strong words used in other computer-related documents, and there are a lot of words on the document that is also used on religion related documents, like 'think', 'good', and 'thing'. We can also see from 2.4 that these words are few of the top 10 word for topic 3. I have tried different n_topics and preprocessing but could not succeed in getting all documents correctly assigned.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. It is a supervised machine learning that suggests item to user based on past user preference and item's feature. This is extremely useful for new items/user. In content-based filtering, we have a utility matrix of users-preferences/ratings, but it is usually very sparse due to missing user preferences on most items. We represent items in a set of features (eg genre). We also create profile for each user, consisting of the item and rating/preference of that user and build a model for each user where the target is the rating/preference of items. The information from the item-feature matrix is then used to recommend new items by comparing the new item's feature with the user profile, and we can complete the utility matrix by predicting rating/preference for each user using their corresponding model. This way, the system can suggest items that are similar to what the user has liked before.
3. - Filter Bubbles: It can create less diversity since the system would recommend stuffs that the user is more likely to like. The user will not likely get any wider exposure to diverse information. This is especially bad in the context of politics and extremism, where it is crucial for one to have a wide range of perspective.
   - Privacy and trust: The system relies on collecting personal data (rating, browsing histrory, purchase history, etc) to be give a personalized recommendation. If not implemented correctly (eg. doesn't ask for consent, not transparent about data usage), it can lead to privacy violations and trust issues.
4. Transfer learning in nlp is the use of pre-trained nlp model that has been trained on a larger dataset, often for a general/wider use, and then fine-tuned to be implemented for a more specific use. This approach is more efficient since it reduces the time and resources needed to train new models

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)